In [ ]:
from itertools import chain
from pathlib import Path

from import_analysis import ImportPatient

# Define paths to patient data.
data_dir = Path("~/data/20220331_import_data_selection").expanduser()
paths = sorted(list(chain(data_dir.glob('import_high/H*'),
                          data_dir.glob('import_low/L*'))))

In [ ]:
# Load data, and calculate dose sums.
patients = []
id = None
for path in paths[0: 1]:
    if id and not path.match(f"*{id}*"):
        continue
    patients.append(ImportPatient(path, load_dose_sum=True, load_masks=False))
                
    print(f"{patients[-1].id} - initialisation; {patients[-1]._init_time:.2f} s; ")

In [ ]:
# Print information, and optionally display images, structure sets, summed dose.
display = True
no_ui = False
view = "y-z"
for patient in patients:
    vmax = 60 if patient.id.startswith("L") else 60

    dose_sum = patient.get_dose_sum()
    ct_plan = patient.get_ct_plan()
    ct_relapse = patient.get_ct_relapse()
    ss_clinical = patient.get_ss_clinical()
    ss_plan = patient.get_ss_plan()
    ss_recurrence = patient.get_ss_recurrence()
    ss_relapse = patient.get_ss_relapse()
    
    print(f"\n{patient.id} - maximum dose: {dose_sum.get_max():.2f} Gy {patient.get_dose_summation_types()}")
    print(f"Plan voxels: {ct_plan.get_n_voxels()}")
    print(f"Plan voxel size: {ct_plan.get_voxel_size()}")
    print(f"Relapse voxels: {ct_relapse.get_n_voxels()}")
    print(f"Relapse voxel size: {ct_relapse.get_voxel_size()}")
    print(f"Plan ROIs: {ss_plan.get_roi_names()}")
    print(f"Relapse ROIs: {ss_relapse.get_roi_names()}")
    print(f"Recurrence ROIs: {ss_recurrence.get_roi_names()}")
    print(f"Clinical ROIs: {ss_clinical.get_roi_names()}")
    
    if display:
        ct_plan.view(images=ct_relapse, init_view=view,
                            annotate_slice="yellow",
                            init_idx=dose_sum.get_centroid_idx(view, 0.9),
                            dose=dose_sum, intensity=(-300, 100), colorbar=-1,
                            colorbar_last_only=False, ytick_labels_first_only=False,
                            dose_kwargs={"vmin": 0, "vmax": vmax},
                            rois=[None, ss_relapse + ss_recurrence], legend=True, no_ui=no_ui)